In [49]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from IPython.display import display, HTML
import seaborn as sns
import random
import matplotlib.pyplot as plt
import plotly.express as px
import pandas_profiling as pp
from scipy import stats
from scipy.stats import norm
%matplotlib inline
import h2o
from h2o.automl import H2OAutoML

In [52]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('data2/landmarks.csv')
df.columns = [f'col{i}' for i in range(1405)]
df.astype({'col1404':'string'})

Y = df.iloc[:]['col1404']
X = df.drop(columns=['col1404'])
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.15)

In [62]:
ytrain.unique()

array(['surprise', 'anger', 'contempt', 'sad', 'happy', 'disgust',
       'neutral', 'fear'], dtype=object)

In [63]:
h2o.init(
    nthreads=8,     # number of threads when launching a new H2O server
    max_mem_size=8  # in gigabytes
)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 hours 29 mins
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_vorkov_s0lk7n
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.731 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [47]:
# train.groupby("col1404")["col1404"].count()

col1404
anger       3159
contempt    2857
disgust        1
happy       5038
neutral     5117
surprise    4019
Name: col1404, dtype: int64

In [ ]:
train = xtrain
train["col1404"] = ytrain

test = xtest
test["col1404"] = ytest

train_h2o_frame = h2o.H2OFrame(train)
test_h2o_frame = h2o.H2OFrame(test)

x = train_h2o_frame.columns
y = "col1404"
x.remove(y)

aml = H2OAutoML(max_models=8, seed=1, max_runtime_secs=60*60, verbosity='info')
aml.train(x=x, y=y, training_frame=train_h2o_frame)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
04:36:41.602: Project: AutoML_2_20230909_43641
04:36:41.602: 5-fold cross-validation will be used.
04:36:41.603: Setting stopping tolerance adaptively based on the training frame: 0.006386420048890296
04:36:41.603: Build control seed: 1
04:36:41.604: training frame: Frame key: AutoML_2_20230909_43641_training_Key_Frame__upload_ae3d0a5414d9ea46ac6abb67ac324507.hex    cols: 1405    rows: 24518  chunks: 157    size: 290473276  checksum: 9014670398134974866
04:36:41.604: validation frame: NULL
04:36:41.604: leaderboard frame: NULL
04:36:41.604: blending frame: NULL
04:36:41.604: response column: col1404
04:36:41.604: fold column: null
04:36:41.604: weights column: null
04:36:41.608: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90

In [24]:
aml.distribution

'AUTO'

In [8]:
lb = aml.leaderboard
lb.head(rows=15)

model_id,mean_per_class_error,logloss,rmse,mse
GLM_1_AutoML_1_20230909_00442,0.358946,0.588139,0.428383,0.183512
XGBoost_1_AutoML_1_20230909_00442,0.384425,0.629443,0.435483,0.189645


In [13]:
test_h2o_frame = test_h2o_frame.drop('col1404')
pred = aml.predict(test_h2o_frame)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [14]:
pred


predict,anger,contempt,disgust,happy,neutral,surprise
anger,0.785474,0.00315324,7.16687e-07,0.000155655,2.75566e-06,0.211214
anger,0.513401,0.224812,1.63442e-05,7.30381e-05,0.00407176,0.257626
anger,0.915206,0.0457143,4.95148e-06,1.73313e-05,0.0040324,0.0350254
anger,0.858818,0.00353934,1.40306e-06,3.76374e-05,1.14827e-05,0.137592
contempt,0.260427,0.48689,2.40392e-05,0.00255855,0.0118256,0.238274
anger,0.732838,0.091804,1.497e-05,7.53426e-05,0.0174777,0.15779
anger,0.828043,0.0637554,3.51999e-07,3.13452e-07,0.000490878,0.10771
anger,0.889751,0.106978,2.23184e-06,5.22006e-05,4.08157e-05,0.00317531
anger,0.56497,0.178844,7.34457e-06,0.000578934,0.038937,0.216663
anger,0.326414,0.152872,5.89797e-05,0.00278635,0.19269,0.325179


In [21]:
best_model = aml.leader
# aml.score(test_h2o_frame, test_h2o_frame.MEDV)


In [23]:
h2o.save_model(model=best_model, path="./")

'/home/vorkov/Workspace/Emotion-Decetion-Service/learning/GLM_1_AutoML_1_20230909_00442'

In [24]:
h2o.load_model('/home/vorkov/Workspace/Emotion-Decetion-Service/learning/GLM_1_AutoML_1_20230909_00442')

Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_1_20230909_00442


GLM Model: summary
    family       link         regularization               lambda_search                                                                   number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  -----------  -----------  ---------------------------  ------------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ---------------------------------------------------------------------------------------
    multinomial  multinomial  Ridge ( lambda = 0.007732 )  nlambda = 30, lambda.max = 21.707, lambda.min = 0.007732, lambda.1se = 0.01062  8430                          8424                           205                     AutoML_1_20230909_00442_training_Key_Frame__upload_9b0ad4f7d1eb0254f20700a6c8f745c0.hex

ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.1832485926285127
RMSE: 0.4280754520274582
LogLoss: 0.5848479289901717
Null degrees of freedom: 20190
Residual degrees of freedom: 11761
Null deviance: 63923.76920676409
Residual deviance: 23617.327619764204
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
anger    contempt    disgust    happy    neutral    surprise    Error      Rate
-------  ----------  ---------  -------  ---------  ----------  ---------  --------------
2146     420         0          18       46         529         0.320671   1,013 / 3,159
429      1948        0          47       84         349         0.318166   909 / 2,857
1        0           0          0        0          0           1          1 / 1
12       23          0          4407     553        43          0.125248   631 / 5,038
46       38          0          316      4614       103         0.0982998  503 / 5,117
383      360         0          104      325        2847        0.291615   1,172 / 4,019
3017     2789        0          4892     5622       3871        0.20945    4,229 / 20,191

Top-6 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.79055
2    0.942301
3    0.984349
4    0.997524
5    0.999851
6    1

ModelMetricsMultinomialGLM: glm
** Reported on cross-validation data. **

MSE: 0.18351216993959682
RMSE: 0.42838320454891415
LogLoss: 0.588139110499819
Null degrees of freedom: 20190
Residual degrees of freedom: 11761
Null deviance: Infinity
Residual deviance: 23750.499560828222
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
anger    contempt    disgust    happy    neutral    surprise    Error      Rate
-------  ----------  ---------  -------  ---------  ----------  ---------  --------------
2125     436         0          17       46         535         0.327319   1,034 / 3,159
407      1972        0          46       84         348         0.309765   885 / 2,857
1        0           0          0        0          0           1          1 / 1
11       22          0          4405     554        46          0.125645   633 / 5,038
46       38          0          316      4610       107         0.0990815  507 / 5,117
389      361         0          100      323        2846        0.291864   1,173 / 4,019
2979     2829 

In [33]:
import numpy as np
from sklearn. metrics import f1_score

pred = aml.predict(test_h2o_frame)
pd=h2o.as_list(pred) 
f1_score(test['col1404'], pd["predict"].tolist(), average="weighted")

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


0.0

In [36]:
test['col1404']

24518     sad
24519     sad
24520     sad
24521     sad
24522     sad
         ... 
28840    fear
28841    fear
28842    fear
28843    fear
28844    fear
Name: col1404, Length: 4327, dtype: object

In [35]:
pd["predict"].tolist()

['anger',
 'anger',
 'anger',
 'anger',
 'contempt',
 'anger',
 'anger',
 'anger',
 'anger',
 'anger',
 'contempt',
 'surprise',
 'surprise',
 'anger',
 'contempt',
 'surprise',
 'contempt',
 'surprise',
 'anger',
 'anger',
 'anger',
 'surprise',
 'surprise',
 'anger',
 'surprise',
 'contempt',
 'anger',
 'anger',
 'contempt',
 'anger',
 'anger',
 'surprise',
 'anger',
 'anger',
 'contempt',
 'anger',
 'surprise',
 'anger',
 'surprise',
 'surprise',
 'anger',
 'anger',
 'anger',
 'anger',
 'contempt',
 'surprise',
 'anger',
 'surprise',
 'surprise',
 'anger',
 'surprise',
 'anger',
 'anger',
 'surprise',
 'neutral',
 'anger',
 'neutral',
 'happy',
 'contempt',
 'anger',
 'contempt',
 'anger',
 'anger',
 'neutral',
 'anger',
 'anger',
 'surprise',
 'surprise',
 'anger',
 'anger',
 'surprise',
 'anger',
 'contempt',
 'contempt',
 'surprise',
 'contempt',
 'surprise',
 'contempt',
 'surprise',
 'surprise',
 'contempt',
 'contempt',
 'happy',
 'anger',
 'surprise',
 'anger',
 'surprise',
 